In [1]:
import os

from dataset import UserGraphDataset
from dataset_in_memory import UserGraphDatasetInMemory
from dataset_in_memory import fetch_question_ids

In [2]:
from hetero_GAT import HeteroGNN
import torch
# Load model (EXP 1)
model = HeteroGNN(64, 2, 3)
model.load_state_dict(torch.load('../models/qa_40e_64h_3l.pt', map_location=torch.device('cpu')))
model.eval()

2023-03-25 18:07:10 INFO     Proceeding with cpu . .


HeteroGNN(
  (convs): ModuleList(
    (0): HeteroConv(num_relations=10)
    (1): HeteroConv(num_relations=10)
    (2): HeteroConv(num_relations=10)
  )
  (lin1): Linear(3392, 2, bias=True)
  (lin2): Linear(64, 2, bias=True)
  (softmax): Softmax(dim=-1)
)

In [3]:
question_ids = fetch_question_ids()
train_ids = list(question_ids)[:int(len(question_ids) * 0.7)]
test_ids = [x for x in question_ids if x not in train_ids]

In [4]:
# Instantiate the dataset class for access to database
ugd = UserGraphDataset('../data/', db_address='../stackoverflow.db', skip_processing=True)

2023-03-25 18:07:11 INFO     PostEmbedding instantiated!


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import torch
import re
import os


def fetch_answer_files(question_id: int):
    for f in os.listdir('../data/processed/'):
        question_id_search = re.search(r"id_(\d+)", f)
        if question_id_search:
            if int(question_id_search.group(1)) == question_id:
                yield torch.load(os.path.join('../data/processed/', f))


In [6]:
from torch_geometric.loader import DataLoader

CORRECT = 0
COUNT = 0
print(len(test_ids))
for i in test_ids[:500]:
    graph_files = list(fetch_answer_files(i))
    dataloader = DataLoader(graph_files, batch_size=1, shuffle=False)

    results = []
    print(i)
    for answer, graph in zip(ugd.fetch_answers_for_question(i).itertuples(), dataloader):

        post_emb = torch.cat([graph.question_emb, graph.answer_emb], dim=1)

        out = model(graph.x_dict, graph.edge_index_dict, graph.batch_dict, post_emb)  # Perform a single forward pass.
        results.append((out[0][1].item(), answer.Score))
    if len(results) > 1:
        # calculate precision@1
        results.sort(key=lambda x: x[0], reverse=True)
        if results[0][1] > max([x[1] for x in results[1:]]):
            #print('Correct')
            CORRECT += 1
        else:
            #print('Incorrect')
            pass
        COUNT += 1



2126
1759455
1464548
2087148
1857780
809210
2218238
1595649
88325
1890570
579856
2054416
1956142
842031
2087216
1530161
973106
1431862
350519
1792312
1169725
252221
874815
2218304
514371
842059
1661262
842075
678236
1661275
153956
1792360
1071469
612734
743806
1235331
1530245
285061
514448
809362
1005972
1235349
2218388
1464728
1956249
940442
481692
1497504
2021803
612788
350647
1563074
2152898
2087370
285132
1268175
1038824
186857
1268209
1759736
252417
842245
1137161
678410
940555
317963
875027
940564
1268252
1563165
449055
186916
88613
875046
1661479
285226
121396
1956407
1038907
1432126
1530430
875074
2087492
383565
350799
1006169
1792602
1464923
875100
2054746
1628766
121439
1825384
285289
1006189
1923706
1464961
1989251
2185862
383623
1235594
613007
2022031
1202839
1825438
1202848
973473
1235618
1727138
1301156
1202855
973481
1235629
1661621
711351
2251450
2316987
2218693
56011
1465036
711376
973520
1006289
875228
482014
1202910
285408
940783
383738
1334020
1956623
285455
1628949

In [7]:
CORRECT

161